In [1]:
import sys
import py_compile
import pandas as pd
import xgboost as xgb
import numpy as np

sys.path.append(r"../")
py_compile.compile("../utility.py")
from utility import rmse
from sklearn.model_selection import train_test_split 

In [2]:
exclude = ['PC', 'SC-TP', 'SST-Indian', 'SST-Pacific', 'AO', 'NAO', 'PNA', 'nina3.4', 'nina1+2', 'nina3', 'nina4']

df = pd.read_csv("../feature-prepare/input-PC2.csv")

features = [feature for feature in list(df.keys()) if feature not in exclude]
target = ['PC']

predPC = []
obsPC = []

idx = [line for line in range(0,32)]
idxx = [line for line in range(32,40)]
train_df = df.iloc[idx][features]
train_target = df.iloc[idx][target]
test_df = df.iloc[idxx][features]
test_target = df.iloc[idxx][target]
    
param = {'nthread':6, #when use hyperthread, xgboost may become slower
              'learning_rate': 0.01, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 1, #5
              'silent': True,
              'subsample': 0.3,
              'colsample_bytree': 1.0,
              'n_estimators': 3000}

dtrain = xgb.DMatrix(train_df, label=train_target, feature_names=features)
dtest  = xgb.DMatrix(test_df, label=test_target, feature_names=features)
    
    
    
cv_res= xgb.cv(param,dtrain,num_boost_round=6000,early_stopping_rounds=3000,nfold=4, metrics='map',show_stdv=False)

bst = xgb.train(param,dtrain,num_boost_round=cv_res.shape[0])
pred = bst.predict(dtest)
    
print(pred)

oo = pd.DataFrame(pred)
oo.to_csv('PC2.txt',header=0,index=0)

[10:50:56] WARNING: ../src/learner.cc:516: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:50:56] WARNING: ../src/learner.cc:516: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:50:56] WARNING: ../src/learner.cc:516: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you 

In [3]:
for impt_type in 'weight', 'gain', 'cover', 'total_gain', 'total_cover':
    print(impt_type + ":        " + str(bst.get_score(importance_type=impt_type)))

weight:        {'SST-Atlantic': 1, 'SC-EU': 1, 'SC-NA': 1, 'SIC': 1}
gain:        {'SST-Atlantic': 1408267.75, 'SC-EU': 527162.688, 'SC-NA': 32713.5547, 'SIC': 217.692871}
cover:        {'SST-Atlantic': 6.0, 'SC-EU': 5.0, 'SC-NA': 3.0, 'SIC': 2.0}
total_gain:        {'SST-Atlantic': 1408267.75, 'SC-EU': 527162.688, 'SC-NA': 32713.5547, 'SIC': 217.692871}
total_cover:        {'SST-Atlantic': 6.0, 'SC-EU': 5.0, 'SC-NA': 3.0, 'SIC': 2.0}
